In [7]:
import pprint
import pymongo
import json
import connector
from dotenv import load_dotenv
load_dotenv()
import os
from pymongo import MongoClient
user = os.getenv("USER")
password = os.getenv("PASSWORD")
client = pymongo.MongoClient("mongodb+srv://"+ user +":"+ password +"@cluster0-doqes.mongodb.net/test?retryWrites=true&w=majority")
db = connector.connect()
collection_trafic = db.vehicules


In [16]:
print(collection_trafic.find().count())

600003


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


In [17]:
pprint.pprint(list(collection_trafic.find()[0:5]))

[{'_id': ObjectId('5ed3ed8fd59082dd24ba102c'),
  'nb_vehicules': 58,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 1,
  'plage_horaire': 'm'},
 {'_id': ObjectId('5ed3ed8fd59082dd24ba102d'),
  'nb_vehicules': 32,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 0,
  'plage_horaire': 'm'},
 {'_id': ObjectId('5ed3ed8fd59082dd24ba102e'),
  'nb_vehicules': 27,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 3,
  'plage_horaire': 'm'},
 {'_id': ObjectId('5ed3ed8fd59082dd24ba102f'),
  'nb_vehicules': 41,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 2,
  'plage_horaire': 'm'},
 {'_id': ObjectId('5ed3ed8fd59082dd24ba1030'),
  'nb_vehicules': 15,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 4,
  'plage_horaire': 'm'}]


In [18]:
collection_trafic.insert_one({"num_arete":501})
print(collection_trafic.find_one({"num_arete":501}))

{'_id': ObjectId('5ed600ab403c8d48c67eee70'), 'num_arete': 501}


In [27]:
collection_trafic.delete_one({"num_arete":501})
print(collection_trafic.find_one({"num_arete":501}))

None


In [25]:
vehicules_par_plage = {"m":0, "s":0}
for trafic in collection_trafic.find({"$or":[{"plage_horaire":"m"}, {"plage_horaire":"s"}]}):
    plage = trafic["plage_horaire"]
    vehicules_par_plage[plage] += trafic["nb_vehicules"]
print(vehicules_par_plage)

{'m': 19720251, 's': 28191853}


In [36]:
vehicules_par_plage = collection_traffic.aggregate
([{$group:{_id: { plage_horaire:  $plage_horaire },
totalAmount: { $sum:  $nb_vehicules}}}])
pprint.pprint(list(vehicules_par_plage))

SyntaxError: invalid syntax (<ipython-input-36-e2454aeecc5d>, line 2)

In [32]:
pprint.pprint(list(collection_trafic.find()[0:5]))

[{'_id': ObjectId('5ed3ed8fd59082dd24ba102c'),
  'nb_vehicules': 58,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 1,
  'plage_horaire': 'm'},
 {'_id': ObjectId('5ed3ed8fd59082dd24ba102d'),
  'nb_vehicules': 32,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 0,
  'plage_horaire': 'm'},
 {'_id': ObjectId('5ed3ed8fd59082dd24ba102e'),
  'nb_vehicules': 27,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 3,
  'plage_horaire': 'm'},
 {'_id': ObjectId('5ed3ed8fd59082dd24ba102f'),
  'nb_vehicules': 41,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 2,
  'plage_horaire': 'm'},
 {'_id': ObjectId('5ed3ed8fd59082dd24ba1030'),
  'nb_vehicules': 15,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 4,
  'plage_horaire': 'm'}]


In [35]:
import datetime

data_stamped = []
for trafic in collection_trafic.find():
    jour= trafic["num_jour"]
    minute= trafic["num_periode"]
    plage= trafic["plage_horaire"]
    date = datetime.date(2020,1,1)
    date.AddDays(jour)
print(date)


AttributeError: 'datetime.date' object has no attribute 'AddDays'